| Teammember |                    |
|------------|--------------------|
| 1.         | Christopher Caldwell |
| 2.         | Fabian Müller      |
| 3.         | An Dang         |

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

tf.reset_default_graph()

In [2]:
from tensorflow.keras.datasets.mnist import load_data
(train_X, train_y),(test_X, test_y) = load_data()

train_X = train_X/255.0
train_y = train_y/255.0

test_X = test_X/255.0
test_y = test_y/255.0


train = np.append(train_X, test_X, axis=0) #Use both Train and Testset for GAN


In [3]:
with tf.name_scope('input'):
    leaky_relu = tf.placeholder(tf.bool, shape=(), name="UseLeakyRelu")

In [4]:
def discriminator(X, reuse_vars=None):
    with tf.variable_scope('dis', reuse=reuse_vars):
        
        
        conv1 = tf.layers.conv2d(inputs=X, filters=32, kernel_size=[5,5], padding="same", name="conv1")
        if leaky_relu is True:
            conv1 = tf.nn.leaky_relu(conv1)
        else:
            conv1 = tf.nn.relu(conv1)
        pool1 = tf.layers.average_pooling2d(inputs=conv1, pool_size=[2,2], strides=2, name="pool1")

        conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[5,5], padding="same", name="conv2")
        
        if leaky_relu is True:
            conv2 = tf.nn.leaky_relu(conv2)
        else:
            conv2 = tf.nn.relu(conv2)
            
        pool2 = tf.layers.average_pooling2d(inputs=conv2, pool_size=[2,2], strides=2, name="pool2")

        flat_layer = tf.reshape(pool2, [-1, 7 * 7 * 64], name="flatten")

        dense = tf.layers.dense(inputs=flat_layer, units=1024, name="dense")
        # Dense layer
        if leaky_relu is True:
            dense = tf.nn.leaky_relu(dense)
        else:
            dense = tf.nn.relu(dense)
        
        
        # Logits layer
        logits = tf.layers.dense(inputs=dense, units=1, name="dense2")
        
        output = tf.sigmoid(logits, name="sigmoid")
        
        return output, logits



In [5]:
def gen(noise, reuse_vars=None):
    with tf.variable_scope("gen", reuse=reuse_vars):
        
        
        
        # First DenseLayer
        dense = tf.layers.dense(inputs=noise, units=3136, name="dense")
        if leaky_relu is True:
            dense = tf.nn.leaky_relu(dense)
        else:
            dense = tf.nn.relu(dense)
            

        #Reshape
        reshape = tf.reshape(dense, [-1, 56, 56, 1], name="reshape")

        
        
        #1 Conv2d
        conv1 = tf.layers.conv2d(inputs=reshape, filters=50, kernel_size=[3,3], strides=2, padding="same", name="conv1")

        if leaky_relu is True:
            conv1 = tf.nn.leaky_relu(conv1)
            
        else:
            conv1 = tf.nn.relu(conv1)
        
        
        #BatchNormalization
        conv1_norm = tf.layers.batch_normalization(conv1, name="batch_norm1")

        #Upsample
        conv1_upsample = tf.image.resize_images(conv1_norm, (56,56))


        #2 Conv2d
        conv2 = tf.layers.conv2d(inputs=conv1_upsample, filters=25, kernel_size=[3,3], strides=2, padding="same", name="conv2")

        if leaky_relu is True:
            conv2 = tf.nn.leaky_relu(conv2)
        else:
            conv2 = tf.nn.relu(conv2)
        
        
        #BatchNormalization
        conv2_norm = tf.layers.batch_normalization(conv2, name="batch_norm2")

        #Upsample
        conv2_upsample = tf.image.resize_images(conv2_norm, (56,56))


        #2 Conv2d
        logits = tf.layers.conv2d(inputs=conv2_upsample, filters=1, kernel_size=[3,3], strides=2, padding="same", name="conv3")

        output = tf.nn.sigmoid(logits, name="sigmoid")
        return output


In [6]:
with tf.name_scope('input'):
    real_images = tf.placeholder(tf.float32, shape=[None, 28,28,1], name="ImagePlaceholder")
    z = tf.placeholder(tf.float32, shape=[None, 100], name = "NoisePlaceholder")

In [7]:
Gz = gen(z)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.batch_normalization instead.


In [8]:
D_output_real, D_logits_real = discriminator(real_images)
D_output_fake, D_logits_fake = discriminator(Gz, reuse_vars=True)

Instructions for updating:
Use keras.layers.average_pooling2d instead.


In [9]:
# LOSSES
def loss_func (logits_in, labels_in):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    logits=logits_in, labels=labels_in, name="CrossEntropy"), name="ReduceMean")

In [10]:
SMOOTHING = tf.placeholder(tf.float32, name = "SmothingFactor")

D_real_loss = loss_func(D_logits_real, tf.ones_like(D_logits_real, name="Generator_Label")*SMOOTHING)
D_fake_loss = loss_func(D_logits_fake, tf.zeros_like(D_logits_fake, name="Discriminator_Label"))

D_loss = D_real_loss + D_fake_loss

In [11]:
G_loss = loss_func(D_logits_fake, tf.ones_like(D_logits_fake, name="Generator_TrickLabel"))

In [12]:
D_learning_rate = tf.placeholder(tf.float32, name = "DiscriminatorLearningRate")
G_learning_rate = tf.placeholder(tf.float32, name = "GeneratorLearningRate")

# learning_rate = 0.001 - worked so far better

In [13]:
d_vars = tf.trainable_variables(scope='dis')
g_vars = tf.trainable_variables(scope='gen')

In [14]:
D_trainer = tf.train.AdamOptimizer(D_learning_rate, name="AdamOptimizer_Discriminator").minimize(D_loss, var_list=d_vars)


Instructions for updating:
Use tf.cast instead.


In [15]:
G_trainer = tf.train.AdamOptimizer(G_learning_rate, name="AdamOptimizer_Discriminator").minimize(G_loss, var_list=g_vars, name="minimize_G_loss")

In [16]:
tf.summary.scalar("Dx_Loss", D_real_loss)
tf.summary.scalar("Dg_Loss", D_fake_loss)
tf.summary.scalar("D_loss", D_loss)
tf.summary.scalar("G_loss", G_loss)

tf.summary.image("Generated_Images", Gz, max_outputs=5)

<tf.Tensor 'Generated_Images:0' shape=() dtype=string>

In [17]:
import datetime

now = datetime.datetime.now()

In [18]:
BATCH_SIZE = tf.placeholder(tf.int64, name = "BatchSizePlaceholder")
#BATCH_SIZE = 100

BUFFER_SIZE = 140000



In [19]:
samples = []
#real = []

In [20]:
#Adding config for GPU - not using all of the GPUs Memory -> leading to crashes
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.95

merged = tf.summary.merge_all()

In [21]:
from tqdm import tqdm
from tensorflow.keras.datasets.mnist import load_data
(train_X, train_y),(test_X, test_y) = load_data()

train_X = train_X/255.0
train_y = train_y/255.0

test_X = test_X/255.0
test_y = test_y/255.0


train = np.append(train_X, test_X, axis=0) #Use both Train and Testset for GAN




batchtrain = train.reshape(train.shape[0], 28, 28, 1).astype('float32')
dataset = tf.data.Dataset.from_tensor_slices(batchtrain).repeat().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
iterator = dataset.make_initializable_iterator()
nextbatch = iterator.get_next()


init = tf.global_variables_initializer()
 

* batch_size:             64 | 128 | 
* learning_rate:          0.0001 | 0.001 | 0.003
* pretrain:               0 |10000 | 20000
* activation function:    tan | sigmoid
* smoothing:              0.9 | 1
* leaky relu:             no | yes

In [22]:
mode_batch_size = [64,128]
mode_learning_rate = [0.0001,0.00001]
mode_disc_lr_multiplier = [1, 3]
mode_pretrain_runs = [0, 10000, 20000]
mode_smoothing = [0.9, 1]
mode_leakyRelu = [False, True]


modes = []

for m_batch_size, m_learning_rate, m_disc_multiplier, m_pretrain, m_smoothing, m_leaky in [(m_batch_size, m_learning_rate, m_disc_multiplier, m_pretrain, m_smoothing, m_leaky) for m_batch_size in mode_batch_size for m_learning_rate in mode_learning_rate for m_disc_multiplier in mode_disc_lr_multiplier for m_pretrain in mode_pretrain_runs for m_smoothing in mode_smoothing for m_leaky in mode_leakyRelu]:
    mode = [m_batch_size, m_learning_rate, m_disc_multiplier, m_pretrain, m_smoothing, m_leaky]
    modes.append(mode)
    
print(len(modes))

96


In [23]:
print(modes)

[[64, 0.0001, 1, 0, 0.9, False], [64, 0.0001, 1, 0, 0.9, True], [64, 0.0001, 1, 0, 1, False], [64, 0.0001, 1, 0, 1, True], [64, 0.0001, 1, 10000, 0.9, False], [64, 0.0001, 1, 10000, 0.9, True], [64, 0.0001, 1, 10000, 1, False], [64, 0.0001, 1, 10000, 1, True], [64, 0.0001, 1, 20000, 0.9, False], [64, 0.0001, 1, 20000, 0.9, True], [64, 0.0001, 1, 20000, 1, False], [64, 0.0001, 1, 20000, 1, True], [64, 0.0001, 3, 0, 0.9, False], [64, 0.0001, 3, 0, 0.9, True], [64, 0.0001, 3, 0, 1, False], [64, 0.0001, 3, 0, 1, True], [64, 0.0001, 3, 10000, 0.9, False], [64, 0.0001, 3, 10000, 0.9, True], [64, 0.0001, 3, 10000, 1, False], [64, 0.0001, 3, 10000, 1, True], [64, 0.0001, 3, 20000, 0.9, False], [64, 0.0001, 3, 20000, 0.9, True], [64, 0.0001, 3, 20000, 1, False], [64, 0.0001, 3, 20000, 1, True], [64, 1e-05, 1, 0, 0.9, False], [64, 1e-05, 1, 0, 0.9, True], [64, 1e-05, 1, 0, 1, False], [64, 1e-05, 1, 0, 1, True], [64, 1e-05, 1, 10000, 0.9, False], [64, 1e-05, 1, 10000, 0.9, True], [64, 1e-05, 1, 1

In [25]:
with tf.Session(config=config) as sess:
    
    for model in modes:   
        sess.run(init)
        
        current_batchSize = model[0]
        current_learningRate = model[1]
        current_discMultiplier = model[2]
        current_pretrainRuns = model[3]
        current_smoothingFactor = model[4]
        current_useLeaky = model[5]
        
        
        EPOCHS = int(5000000/current_batchSize)
        
        model_text = "{0}-batchsize_{1}-lr_{2}-discMultiplier_{3}-pretrainRuns_{4}-smoothing_{5}-leakyRelu".format(
            current_batchSize, current_learningRate, current_discMultiplier, current_pretrainRuns, current_smoothingFactor, current_useLeaky)
        
        # Naming the different Tensorboards. Additionally to the date, we added a shot description of the current model
        writer = tf.summary.FileWriter('./tensorboard/{0}_GAN-{1}'.format(now.strftime("%Y_%m_%d-%H-%M"), model_text), sess.graph)

        
        
        
        
        
        # Pretrain Epochs for Discriminator. For 20000 images we need 200 Epochs with a batch size of 100
        for pretrain in tqdm(range(int( current_pretrainRuns/current_batchSize))):
            sess.run(iterator.initializer, feed_dict={BATCH_SIZE:current_batchSize})
            batch_images = sess.run(nextbatch)

            # We used random normal distribution with standard deviation of 0.5 -> Explanation below
            batch_z =  np.random.normal(0.0, 1, size=(current_batchSize,100))

            #if current_useLeaky:
            _ = sess.run(D_trainer, feed_dict={real_images:batch_images, z:batch_z, D_learning_rate:current_learningRate*current_discMultiplier, SMOOTHING: current_smoothingFactor, leaky_relu:current_useLeaky})

            
            #else:
            #    _ = sess.run(D_trainer, feed_dict={real_images:batch_images, z:batch_z, D_learning_rate:current_learningRate*current_discMultiplier, SMOOTHING: current_smoothingFactor})


        for epoch in tqdm(range(EPOCHS)):
            sess.run(iterator.initializer, feed_dict={BATCH_SIZE:current_batchSize})
            batch_images = sess.run(nextbatch, feed_dict={BATCH_SIZE:current_batchSize})

            # We used random normal distribution with standard deviation of 0.5 -> Explanation below
            batch_z = np.random.normal(0.0, 1, size=(current_batchSize,100))

            #if current_useLeaky:
            D_summary, _, discrloss, discr_r_loss, discr_f_loss = sess.run([merged, D_trainer, D_loss, D_real_loss, D_fake_loss], feed_dict={real_images:batch_images, z:batch_z, D_learning_rate:current_learningRate*current_discMultiplier, SMOOTHING: current_smoothingFactor, leaky_relu:current_useLeaky})
            
            #else:
            #    D_summary, _, discrloss, discr_r_loss, discr_f_loss = sess.run([merged, D_trainer, D_loss, D_real_loss, D_fake_loss], feed_dict={real_images:batch_images, z:batch_z, D_learning_rate:current_learningRate*current_discMultiplier, SMOOTHING: current_smoothingFactor})

            genloss, _ = sess.run([G_loss, G_trainer], feed_dict={z:batch_z, G_learning_rate:current_learningRate, leaky_relu:current_useLeaky})

            if (epoch % 10 == 0):{
                # SUMMARIES
                writer.add_summary(D_summary, epoch)      
            }


            if (EPOCHS%4000 == 0):
                # We used random normal distribution with standard deviation of 0.5 -> Explanation below
                #sample_z = np.random.uniform(0,1,size=(1,100))
                sample_z = np.random.normal(0.0, 1, size=(1,100))
                
                # Print out all losses
                print("Discriminator-Real-Loss:", discr_r_loss)

                print("Discriminator-Fake-Loss:", discr_f_loss)

                print("Discriminator Loss:",discrloss)

                print("Generator Loss:",genloss)

                # Generate a sample image
                gen_sample = sess.run(gen(z, reuse_vars=True), feed_dict={z: sample_z, leaky_relu:current_useLeaky})

                # Display the sample image at current epoch
                plt.imshow(gen_sample.reshape(28,28), cmap="gray")
                plt.show()




0it [00:00, ?it/s]
  0%|                                                                            | 67/78125 [01:00<16:23:31,  1.32it/s]


  0%|▏                                                                          | 135/78125 [01:52<16:56:08,  1.28it/s]


  0%|▏                                                                          | 203/78125 [02:43<16:12:39,  1.34it/s]


  0%|▎                                                                          | 271/78125 [03:35<16:13:04,  1.33it/s]


  0%|▎                                                                          | 293/78125 [03:53<17:21:18,  1.25it/s]

KeyboardInterrupt: 